# Lesson 4 WM

## To Do
- set up notebook
- set up data
- build model
- train model

## Set up notebook

In [1]:
from theano.sandbox import cuda

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using Theano backend.


In [3]:
os.getcwd()

'/home/ubuntu/nbs'

In [138]:
#path = 'data/ml-small/'
path = 'data/ml-20m/'
model_path = path + 'models/'
if not os.path.exists(path): os.mkdir(path)
if not os.path.exists(model_path): os.mkdir(model_path)

In [139]:
batch_size = 128

## Set up data

In [140]:
ratings = pd.read_csv(path + 'ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [141]:
len(ratings)

20000263

In [142]:
movie_names = pd.read_csv(path + 'movies.csv')
movie_names.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [143]:
len(movie_names)

27278

In [144]:
n_users = ratings.userId.nunique()
u_users = ratings.userId.unique()
userid2idx = {u:idx for idx,u in enumerate(u_users)}

n_movies = ratings.movieId.nunique()
u_movies = ratings.movieId.unique()
movieid2idx = {m:idx for idx,m in enumerate(u_movies)}
n_users, n_movies

(138493, 26744)

In [145]:
ratings.userId = ratings.userId.apply(lambda x: userid2idx[x])
ratings.movieId = ratings.movieId.apply(lambda x: movieid2idx[x])

In [146]:
user_min, user_max, movie_min, movie_max = (ratings.userId.min(), ratings.userId.max(),
                                           ratings.movieId.min(), ratings.movieId.max())
user_min, user_max, movie_min, movie_max

(0, 138492, 0, 26743)

In [147]:
np.random.seed = 42

In [148]:
mask = np.random.rand(len(ratings)) < 0.8
train = ratings[mask]
valid = ratings[~mask]
len(train), len(valid)

(15999772, 4000491)

## Build model

In [149]:
user_inp = Input(shape=(1,))
movie_inp = Input(shape=(1,))
n_factors = 100

u_emb = Embedding(n_users, n_factors,input_length=1,W_regularizer=l2(1e-2))(user_inp)
m_emb = Embedding(n_movies, n_factors,input_length=1,W_regularizer=l2(1e-2))(movie_inp)
# where is the bias accounted for?

In [150]:
x = merge([u_emb, m_emb], mode='concat')
x = Flatten()(x)
x = Dropout(0.2)(x)
x = Dense(150, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(100, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(50, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(10, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(1, activation='relu')(x)

In [151]:
model = Model(input=[user_inp, movie_inp],output=x)
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_11 (InputLayer)            (None, 1)             0                                            
____________________________________________________________________________________________________
input_12 (InputLayer)            (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_11 (Embedding)         (None, 1, 100)        13849300    input_11[0][0]                   
____________________________________________________________________________________________________
embedding_12 (Embedding)         (None, 1, 100)        2674400     input_12[0][0]                   
___________________________________________________________________________________________

## Train model

In [152]:
model.compile(optimizer=Adam(lr=1e-4),loss='mse')

In [153]:
model.fit([train.userId, train.movieId], train.rating, batch_size=batch_size, nb_epoch=1,
         validation_data=([valid.userId, valid.movieId], valid.rating))

Train on 15999772 samples, validate on 4000491 samples
Epoch 1/1
  194816/15999772 [..............................] - ETA: 5533s - loss: 17.9248

KeyboardInterrupt: 

In [154]:
model.optimizer.lr.set_value(1e-3)

In [155]:
model.fit([train.userId, train.movieId], train.rating, batch_size=batch_size, nb_epoch=1,
         validation_data=([valid.userId, valid.movieId], valid.rating))

Train on 15999772 samples, validate on 4000491 samples
Epoch 1/1
  115840/15999772 [..............................] - ETA: 5799s - loss: 2.5409

KeyboardInterrupt: 

In [156]:
model.optimizer.lr.set_value(1e-4)
model.fit([train.userId, train.movieId], train.rating, batch_size=batch_size, nb_epoch=1,
         validation_data=([valid.userId, valid.movieId], valid.rating))

Train on 15999772 samples, validate on 4000491 samples
Epoch 1/1
  215424/15999772 [..............................] - ETA: 5664s - loss: 1.9367

KeyboardInterrupt: 

In [157]:
model.optimizer.lr.set_value(1e-5)
model.fit([train.userId, train.movieId], train.rating, batch_size=batch_size, nb_epoch=1,
         validation_data=([valid.userId, valid.movieId], valid.rating))

Train on 15999772 samples, validate on 4000491 samples
Epoch 1/1
  114432/15999772 [..............................] - ETA: 5706s - loss: 1.8155

KeyboardInterrupt: 

In [158]:
model.save_weights(model_path + 'wm_nn.h5')